<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab 3_01: Statistical Modeling and Model Validation

> Authors: Tim Book, Matt Brems

---

## Objective
The goal of this lab is to guide you through the modeling workflow to produce the best model you can. In this lesson, you will follow all best practices when slicing your data and validating your model. 

You will be predicting `tripduration`.
Take note the trip duration was recorded in seconds.

## Imports

In [ ]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
# Muscle memory is important!

import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

import seaborn               as sns
import matplotlib.pyplot     as plt

import statsmodels.api as sm

%matplotlib inline

## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [4]:
# Read in the citibike data in the data folder in this repository.
citibike = pd.read_csv('data/citibike_feb2014.csv')

## Explore the data
Use this space to familiarize yourself with the data.

Convince yourself there are no issues with the data. If you find any issues, clean them here.

You should minimally check for:
* Is there missing data?
* Is there duplicated data entries?
* Is there odd data value (e.g. Negative Age value)?
* Is the data type appropriate?

In [5]:
#Check if data is empty & address them appropriately
missing_values = citibike.isnull().sum()

print("Missing values DataFrame :")
print(missing_values)

Missing values DataFrame :
tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64


In [6]:
#Check if data is empty & address them appropriately
# Check if the DataFrame is empty
if citibike.empty:
    # Handle the case where the DataFrame is empty
    print("The DataFrame is empty. No data to work with.")
else:
    # If the DataFrame is not empty, you can perform further operations
    # For example, display the first few rows
    print("The DataFrame is not empty. Here are the first 5 rows:")
    print(citibike.head())

The DataFrame is not empty. Here are the first 5 rows:
   tripduration            starttime             stoptime  start station id  \
0           382  2014-02-01 00:00:00  2014-02-01 00:06:22               294   
1           372  2014-02-01 00:00:03  2014-02-01 00:06:15               285   
2           591  2014-02-01 00:00:09  2014-02-01 00:10:00               247   
3           583  2014-02-01 00:00:32  2014-02-01 00:10:15               357   
4           223  2014-02-01 00:00:41  2014-02-01 00:04:24               401   

        start station name  start station latitude  start station longitude  \
0      Washington Square E               40.730494               -73.995721   
1       Broadway & E 14 St               40.734546               -73.990741   
2   Perry St & Bleecker St               40.735354               -74.004831   
3       E 11 St & Broadway               40.732618               -73.991580   
4  Allen St & Rivington St               40.720196               -73.989978

In [7]:
#check if there is duplicated data & address them appropriately

# Check for duplicated rows
duplicates = citibike[citibike.duplicated(keep='first')]

if not duplicates.empty:
    # If there are duplicated rows, you can either remove them or handle them as needed.
    citibike = citibike.drop_duplicates(keep='first')

    print("Duplicated rows:")
    print(duplicates)

else:
    print("No duplicated rows found.")

No duplicated rows found.


In [8]:
#Is there odd data value (e.g. Negative Age value)?
negative_tripduration_rows = citibike[citibike['tripduration'] < 0]

if not negative_tripduration_rows.empty:
    # Handle the case of negative tripduration values
    print("Rows with negative tripduration values:")
    print(negative_tripduration_rows)
else:
    print("No negative tripduration values found.")

No negative tripduration values found.


In [9]:
#Check datatypes & convert columns data types appropriately
# Check the current data types of columns
print("Current data types:")
print(citibike.dtypes)


# Convert 'starttime' and 'stoptime' columns to datetime
citibike['starttime'] = pd.to_datetime(citibike['starttime'])
citibike['stoptime'] = pd.to_datetime(citibike['stoptime'])

# Check the updated data types
print("\nUpdated data types:")
print(citibike.dtypes)

Current data types:
tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                  object
gender                       int64
dtype: object

Updated data types:
tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int64
end station name                   object
end station latitude              float64


In [10]:
#Check non-numerical columns data:
# Get a list of non-numeric columns
non_numeric_columns = citibike.select_dtypes(exclude=[int, float]).columns

# Print the list of non-numeric columns
print("Non-numeric columns:")
print(non_numeric_columns)

Non-numeric columns:
Index(['starttime', 'stoptime', 'start station name', 'end station name',
       'usertype', 'birth year'],
      dtype='object')


In [11]:
# Check numerical data values (excluding ID columns):

exclude_columns = ['bikeid']

# Get a list of numerical columns (excluding the ID columns)
numerical_columns = [col for col in citibike.select_dtypes(include=[int, float]).columns if col not in exclude_columns]

# Print the numerical data values in these columns
numerical_data = citibike[numerical_columns]
print("Numerical data values (excluding ID columns):")
print(numerical_data)

Numerical data values (excluding ID columns):
        tripduration  start station id  start station latitude  \
0                382               294               40.730494   
1                372               285               40.734546   
2                591               247               40.735354   
3                583               357               40.732618   
4                223               401               40.720196   
...              ...               ...                     ...   
224731           848               498               40.748549   
224732          1355               470               40.743453   
224733           304               497               40.737050   
224734           308               353               40.685396   
224735           603               252               40.732264   

        start station longitude  end station id  end station latitude  \
0                    -73.995721             265             40.722293   
1              

In [12]:
# Examine 'tripduration' - is there outlier?
# Define a function to remove outliers based on the IQR
def remove_outliers_iqr(data, column_name):
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column_name] >= lower_bound) & (data[column_name] <= upper_bound)]

# Remove outliers from 'tripduration'
citibike_cleaned = remove_outliers_iqr(citibike, 'tripduration')

# Print the number of rows before and after removing outliers
print("Number of rows before removing outliers:", len(citibike))
print("Number of rows after removing outliers:", len(citibike_cleaned))

Number of rows before removing outliers: 224736
Number of rows after removing outliers: 210163


In [13]:
# Remove rows where 'birth year' is not a number
citibike_cleaned = citibike_cleaned[pd.to_numeric(citibike_cleaned['birth year'], errors='coerce').notnull()]

# Reset the DataFrame index after removing rows
citibike_cleaned.reset_index(drop=True, inplace=True)

# Print the DataFrame after removing non-numeric 'birth year'
print("Number of rows after removing non-numeric 'birth year:", len(citibike_cleaned))

Number of rows after removing non-numeric 'birth year: 205548


## Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

$$
\begin{eqnarray*}
&H_0:& \mu_1 = \mu_2 \\
&H_A:& \mu_1 \neq \mu_2
\end{eqnarray*}
$$

We will conduct this test assuming $\alpha=0.05$.

In [15]:
from scipy import stats

In [16]:
# Separate data into two groups based on gender
trip_duration_gender1 = citibike[citibike['gender'] == 1]['tripduration']
trip_duration_gender2 = citibike[citibike['gender'] == 2]['tripduration']

t_stat, p_value = stats.ttest_ind(trip_duration_gender1, trip_duration_gender2, equal_var=False)

alpha = 0.05

if p_value < alpha:
    print("Reject the null hypothesis. The average trip duration is different for gender=1 and gender=2.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in average trip duration for gender=1 and gender=2.")

Reject the null hypothesis. The average trip duration is different for gender=1 and gender=2.


In [40]:
print(p_value)
print(alpha)

1.5680482053980366e-06
0.05


**Answer**: <br>

We default to acceptable (alpha) 0.05.
We therefore conclude that the results of this test reject H0 and accept H1.


## What numeric columns shouldn't be treated as numeric?

**Answer:**
start station id

end station id

bikeid

gender

## Dummify the `start station id` Variable

In [18]:
citibike_cleaned_dummies = pd.get_dummies(citibike_cleaned, columns=['start station id'], prefix='start_station', drop_first=True)
citibike_cleaned_dummies

,tripduration,starttime,stoptime,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,...,start_station_2006,start_station_2008,start_station_2009,start_station_2010,start_station_2012,start_station_2017,start_station_2021,start_station_2022,start_station_2023,start_station_3002
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,...,0,0,0,0,0,0,0,0,0,0
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,-73.989780,...,0,0,0,0,0,0,0,0,0,0
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,-73.994800,...,0,0,0,0,0,0,0,0,0,0
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,...,0,0,0,0,0,0,0,0,0,0
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,-73.989780,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205543,848,2014-02-28 23:57:13,2014-03-01 00:11:21,Broadway & W 32 St,40.748549,-73.988084,432,E 7 St & Avenue A,40.726218,-73.983799,...,0,0,0,0,0,0,0,0,0,0
205544,1355,2014-02-28 23:57:55,2014-03-01 00:20:30,W 20 St & 8 Ave,40.743453,-74.000040,302,Avenue D & E 3 St,40.720828,-73.977932,...,0,0,0,0,0,0,0,0,0,0
205545,304,2014-02-28 23:58:17,2014-03-01 00:03:21,E 17 St & Broadway,40.737050,-73.990093,334,W 20 St & 7 Ave,40.742388,-73.997262,...,0,0,0,0,0,0,0,0,0,0
205546,308,2014-02-28 23:59:10,2014-03-01 00:04:18,S Portland Ave & Hanson Pl,40.685396,-73.974315,365,Fulton St & Grand Ave,40.682232,-73.961458,...,0,0,0,0,0,0,0,0,0,0


## Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected).

- Note: you will need to clean the data a bit if you have not done a proper job previously.

In [19]:
# Data Cleaning: Check for missing or incorrect 'birth year' values
citibike_cleaned_dummies['birth year'] = pd.to_numeric(citibike_cleaned_dummies['birth year'], errors='coerce')

# Calculate 'age' in 2014
citibike_cleaned_dummies['age'] = 2014 - citibike_cleaned_dummies['birth year']

# Drop rows with missing or incorrect birth year values
citibike_cleaned_dummies = citibike_cleaned_dummies.dropna(subset=['age'])

# Reset the DataFrame index after removing rows
citibike_cleaned_dummies.reset_index(drop=True, inplace=True)

print(len(citibike_cleaned_dummies))

205548


## Split your data into train/test data

Look at the size of your data. What is a good proportion for your split? **Justify your answer.**

Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `gender`, and the dummy variables you created from `start station id`.

**NOTE:** When doing your train/test split, please use random seed 123.

In [20]:
# Specify the features (X) and target variable (y)

x = citibike_cleaned_dummies[['age','gender']]
start_station = [column for column in citibike_cleaned_dummies.columns if column.startswith('start_station')]
x_with_start_station = citibike_cleaned_dummies[start_station]
x = pd.concat([x, x_with_start_station], axis=1)
y = citibike_cleaned_dummies['tripduration']

# Split the data into training and testing sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

# Check the size of the training and testing sets
print("Training data size:", x_train.shape[0])
print("Testing data size:", x_train.shape[0])


Training data size: 164438
Testing data size: 164438


## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

In [23]:
# Step 1. Instantiate the model.
model = LinearRegression()

# Step 2. Fit the model on the training data.
model.fit(x_train, y_train)

# Step 3. Generate predictions.
y_train_preds = model.predict(x_train)
y_test_preds = model.predict(x_test)

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ (prediction) to be the mean of our training `y` values.)

In [24]:
# Check the MSE on the training and testing sets.
# Check the MSE on the training and testing sets.
mse_train = mean_squared_error(y_train, y_train_preds)

# Calculate MSE for the testing set
mse_test = mean_squared_error(y_test, y_test_preds)

# Print the MSE for both sets
print(f"Mean Squared Error (MSE) for Training Data: {mse_train:.2f}")
print(f"Mean Squared Error (MSE) for Testing Data: {mse_test:.2f}")


Mean Squared Error (MSE) for Training Data: 94695.05
Mean Squared Error (MSE) for Testing Data: 94553.66


In [25]:
# Check the R^2 on the training and testing sets.

r2_train = r2_score(y_train, y_train_preds)

# Calculate R-squared (R²) for the testing set
r2_test = r2_score(y_test, y_test_preds)

# Print the R² for both sets
print(f"R-squared (R²) for Training Data: {r2_train:.2f}")
print(f"R-squared (R²) for Testing Data: {r2_test:.2f}")


R-squared (R²) for Training Data: 0.05
R-squared (R²) for Testing Data: 0.05


**Answer**:  



## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

In [29]:
# Remember, we need to add a constant in statsmodels!
x = citibike_cleaned_dummies[['age','gender']]
start_station = [column for column in citibike_cleaned_dummies.columns if column.startswith('start_station')]
x_with_start_station = citibike_cleaned_dummies[start_station]
x = pd.concat([x, x_with_start_station], axis=1)
y = citibike_cleaned_dummies['tripduration']

# Add a constant to the features (intercept term)
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

# Print a summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           tripduration   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     32.59
Date:                Thu, 19 Oct 2023   Prob (F-statistic):               0.00
Time:                        13:34:57   Log-Likelihood:            -1.4692e+06
No. Observations:              205548   AIC:                         2.939e+06
Df Residuals:                  205217   BIC:                         2.942e+06
Df Model:                         330                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                583.7066     13

## Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

In [31]:
# Remember, we need to add a constant in statsmodels!
x = citibike_cleaned_dummies[['age','gender']]
start_station = [column for column in citibike_cleaned_dummies.columns if column.startswith('start_station')]
x_with_start_station = citibike_cleaned_dummies[start_station]
x = pd.concat([x, x_with_start_station], axis=1)
y = citibike_cleaned_dummies['tripduration']

# Add a constant to the features (intercept term)
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

summary = model.summary()
# Extract the p-value for the 'age' coefficient
p_value_age = model.pvalues['age']

# Define the significance level (alpha)
alpha = 0.05

# Print the summary and perform the hypothesis test
# print(summary)

if p_value_age < alpha:
    print("Age has a significant effect on 'tripduration'.")
else:
    print("Age does not have a significant effect on 'tripduration'.")


Age has a significant effect on 'tripduration'.


In [32]:
# Remember, we need to add a constant in statsmodels!
x = citibike_cleaned_dummies[['age','gender']]
start_station = [column for column in citibike_cleaned_dummies.columns if column.startswith('start_station')]
x_with_start_station = citibike_cleaned_dummies[start_station]
x = pd.concat([x, x_with_start_station], axis=1)
y = citibike_cleaned_dummies['tripduration']

# Add a constant to the features (intercept term)
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

summary = model.summary()
# Extract the p-value for the 'gender' coefficient
p_value_gender = model.pvalues['gender']

# Define the significance level (alpha)
alpha = 0.05

# Print the summary and perform the hypothesis test
# print(summary)

if p_value_gender < alpha:
    print("gender has a significant effect on 'tripduration'.")
else:
    print("gender does not have a significant effect on 'tripduration'.")



gender has a significant effect on 'tripduration'.


$$
\begin{eqnarray*}
&H_0:& \beta_{age} = 0 \\
&H_A:& \beta_{age} \neq 0
\end{eqnarray*}
$$

We will conduct this test assuming $\alpha=0.05$.

**Answer**: 

## Citi Bike is attempting to market to people who they think will ride their bike for a long time. Based on your modeling, what types of individuals should Citi Bike market toward?

In [33]:

port_survival = citibike_cleaned_dummies.groupby('gender')['tripduration'].mean()

print("gender:")
print(port_survival)

gender:
gender
0    387.000000
1    569.586989
2    641.116075
Name: tripduration, dtype: float64


In [35]:
citibike_cleaned_dummies['AgeGroup'] = pd.cut(citibike_cleaned_dummies['age'], bins=[0, 12, 18, 30, 50, 100], labels=['Child', 'Teen', 'Young Adult', 'Adult', 'Senior'])

ageGroup_tripduration = citibike_cleaned_dummies.groupby('AgeGroup')['tripduration'].mean()

print("Tripduration mean by Age Group:")
print(ageGroup_tripduration)

Tripduration mean by Age Group:
AgeGroup
Child                 NaN
Teen           573.663004
Young Adult    565.310428
Adult          582.574506
Senior         614.254143
Name: tripduration, dtype: float64


**Answer:** 

